In [ ]:
import re

In [ ]:
#names = ['atoB', 'Hbd1', 'Crt1', 'Ter1', 'adhE1']
names = ['rfp']
sequences = []
for name in names:
    with open('../genes/{0}.txt'.format(name.lower()), 'r') as f:
        sequences.append(f.read().strip())

In [ ]:
def ouput_transcription_rxns(protein_name, gene_sequence, transcription_template):
    nt_cnts = Counter(gene_sequence.upper())
    txn_template = transcription_template.replace('<PROTEIN>', protein_name)
    for nt in ['A', 'C', 'T', 'G']:
        txn_template = txn_template.replace('<NUM_{0}>'.format(nt), str(nt_cnts[nt]))
    txn_template = txn_template.replace('<NUM_NT>', str(sum(nt_cnts.values())))
    return txn_template

def output_translation_rxns(protein_name, gene_sequence, translation_template, aa_sequence=None):
    seq = Seq(gene_sequence)
    prot_seq = seq.transcribe().translate()
    aa_cnts = Counter(prot_seq)
    tln_template = translation_template.replace('<PROTEIN>', protein_name)
    for aa in Bio.PDB.Polypeptide.aa3:
        tln_template = tln_template.replace('<NUM_{0}>'.format(aa), str(aa_cnts[Bio.PDB.Polypeptide.three_to_one(aa)]))
    n_aas = sum(aa_cnts.values())
    tln_template = tln_template.replace('<NUM_AA>', str(n_aas))
    tln_template = tln_template.replace('<2*NUM_AA>', str(2*n_aas))
    return tln_template

def output_trna_charging_rxns(pathway_name, gene_sequences, trna_charging_template, aa_sequences=None):
    aa_cnts = [Counter(Seq(gene_sequence).transcribe().translate()) for gene_sequence in gene_sequences]
    aa_tot_cnts = reduce(lambda cntr, acc: cntr + acc, aa_cnts)
    trc_template = trna_charging_template.replace('<PATHWAY>', pathway_name)
    for aa in Bio.PDB.Polypeptide.aa3:
        trc_template = trc_template.replace('<NUM_{0}>'.format(aa), str(aa_tot_cnts[Bio.PDB.Polypeptide.three_to_one(aa)]))
    return trc_template

In [ ]:
protein = 'RFP'
with open('../3rd_Party_Code/Sequence-Specific-FBA-CFPS-Publication-Code/Template/Reaction_Base.net', 'r') as f_r, open('../3rd_Party_Code/Sequence-Specific-FBA-CFPS-Publication-Code/Template/Reactions_full.net', 'w') as f_w:
    for line_raw in f_r.readlines():
        line = line_raw.strip()
        # Comments
        #if re.match(r'^(//|#)', line):
        #    continue
        # Special lines
        if re.match(r'.*<.*', line):
            if line.startswith('transcription') or line.startswith('mRNA_degradation'):
                out_line = ouput_transcription_rxns(protein, sequences[0], line_raw)
            elif line.startswith('translation'):
                out_line = output_translation_rxns(protein, sequences[0], line_raw)
            elif line.startswith('tRNA_c_charging'):
                out_line = output_trna_charging_rxns(protein, sequences, line_raw)
            # Protein export
            else:
                out_line = line_raw.replace('<PROTEIN>', protein)
            f_w.write(out_line)
        else:
            f_w.write(line_raw)

In [ ]:
transcription_template = """transcriptional_initiation_<PROTEIN>,GENE_<PROTEIN>+RNAP,OPEN_GENE_<PROTEIN>,0,inf
transcription_<PROTEIN>,OPEN_GENE_<PROTEIN>+<NUM_G>*M_gtp_c+<NUM_C>*M_ctp_c+<NUM_T>*M_utp_c+<NUM_A>*M_atp_c+<NUM_NT>*M_h2o_c,mRNA_<PROTEIN>+GENE_<PROTEIN>+RNAP+<NUM_NT>*M_ppi_c,0,inf
mRNA_degradation_<PROTEIN>,mRNA_<PROTEIN>,<NUM_G>*M_gmp_c+<NUM_C>*M_cmp_c+<NUM_T>*M_ump_c+<NUM_A>*M_amp_c,0,inf"""

In [ ]:
translation_template = """translation_initiation_<PROTEIN>,mRNA_<PROTEIN>+RIBOSOME,RIBOSOME_START_<PROTEIN>,0,inf
translation_<PROTEIN>,RIBOSOME_START_<PROTEIN>+<2*NUM_AA>*M_gtp_c+<2*NUM_AA>*M_h2o_c+<NUM_ALA>*M_ala_L_c_tRNA_c+<NUM_ARG>*M_arg_L_c_tRNA_c+<NUM_ASN>*M_asn_L_c_tRNA_c+<NUM_ASP>*M_asp_L_c_tRNA_c+<NUM_CYS>*M_cys_L_c_tRNA_c+<NUM_GLU>*M_glu_L_c_tRNA_c+<NUM_GLN>*M_gln_L_c_tRNA_c+<NUM_GLY>*M_gly_L_c_tRNA_c+<NUM_HIS>*M_his_L_c_tRNA_c+<NUM_ILE>*M_ile_L_c_tRNA_c+<NUM_LEU>*M_leu_L_c_tRNA_c+<NUM_LYS>*M_lys_L_c_tRNA_c+<NUM_MET>*M_met_L_c_tRNA_c+<NUM_PHE>*M_phe_L_c_tRNA_c+<NUM_PRO>*M_pro_L_c_tRNA_c+<NUM_SER>*M_ser_L_c_tRNA_c+<NUM_THR>*M_thr_L_c_tRNA_c+<NUM_TRP>*M_trp_L_c_tRNA_c+<NUM_TYR>*M_tyr_L_c_tRNA_c+<NUM_VAL>*M_val_L_c_tRNA_c,RIBOSOME+mRNA_<PROTEIN>+PROTEIN_<PROTEIN>+<2*NUM_AA>*M_gdp_c+<2*NUM_AA>*M_pi_c+<NUM_AA>*tRNA_c,0,inf"""

In [ ]:
trna_charging_template = """tRNA_c_charging_M_ala_L_c_<PATHWAY>,<NUM_ALA>*M_ala_L_c+<NUM_ALA>*M_atp_c+<NUM_ALA>*tRNA_c+<NUM_ALA>*M_h2o_c,<NUM_ALA>*M_ala_L_c_tRNA_c+<NUM_ALA>*M_amp_c+<NUM_ALA>*M_ppi_c,0,inf
tRNA_c_charging_M_arg_L_c_<PATHWAY>,<NUM_ARG>*M_arg_L_c+<NUM_ARG>*M_atp_c+<NUM_ARG>*tRNA_c+<NUM_ARG>*M_h2o_c,<NUM_ARG>*M_arg_L_c_tRNA_c+<NUM_ARG>*M_amp_c+<NUM_ARG>*M_ppi_c,0,inf
tRNA_c_charging_M_asn_L_c_<PATHWAY>,<NUM_ASN>*M_asn_L_c+<NUM_ASN>*M_atp_c+<NUM_ASN>*tRNA_c+<NUM_ASN>*M_h2o_c,<NUM_ASN>*M_asn_L_c_tRNA_c+<NUM_ASN>*M_amp_c+<NUM_ASN>*M_ppi_c,0,inf
tRNA_c_charging_M_asp_L_c_<PATHWAY>,<NUM_ASP>*M_asp_L_c+<NUM_ASP>*M_atp_c+<NUM_ASP>*tRNA_c+<NUM_ASP>*M_h2o_c,<NUM_ASP>*M_asp_L_c_tRNA_c+<NUM_ASP>*M_amp_c+<NUM_ASP>*M_ppi_c,0,inf
tRNA_c_charging_M_cys_L_c_<PATHWAY>,<NUM_CYS>*M_cys_L_c+<NUM_CYS>*M_atp_c+<NUM_CYS>*tRNA_c+<NUM_CYS>*M_h2o_c,<NUM_CYS>*M_cys_L_c_tRNA_c+<NUM_CYS>*M_amp_c+<NUM_CYS>*M_ppi_c,0,inf
tRNA_c_charging_M_glu_L_c_<PATHWAY>,<NUM_GLU>*M_glu_L_c+<NUM_GLU>*M_atp_c+<NUM_GLU>*tRNA_c+<NUM_GLU>*M_h2o_c,<NUM_GLU>*M_glu_L_c_tRNA_c+<NUM_GLU>*M_amp_c+<NUM_GLU>*M_ppi_c,0,inf
tRNA_c_charging_M_gln_L_c_<PATHWAY>,<NUM_GLN>*M_gln_L_c+<NUM_GLN>*M_atp_c+<NUM_GLN>*tRNA_c+<NUM_GLN>*M_h2o_c,<NUM_GLN>*M_gln_L_c_tRNA_c+<NUM_GLN>*M_amp_c+<NUM_GLN>*M_ppi_c,0,inf
tRNA_c_charging_M_gly_L_c_<PATHWAY>,<NUM_GLY>*M_gly_L_c+<NUM_GLY>*M_atp_c+<NUM_GLY>*tRNA_c+<NUM_GLY>*M_h2o_c,<NUM_GLY>*M_gly_L_c_tRNA_c+<NUM_GLY>*M_amp_c+<NUM_GLY>*M_ppi_c,0,inf
tRNA_c_charging_M_his_L_c_<PATHWAY>,<NUM_HIS>*M_his_L_c+<NUM_HIS>*M_atp_c+<NUM_HIS>*tRNA_c+<NUM_HIS>*M_h2o_c,<NUM_HIS>*M_his_L_c_tRNA_c+<NUM_HIS>*M_amp_c+<NUM_HIS>*M_ppi_c,0,inf
tRNA_c_charging_M_ile_L_c_<PATHWAY>,<NUM_ILE>*M_ile_L_c+<NUM_ILE>*M_atp_c+<NUM_ILE>*tRNA_c+<NUM_ILE>*M_h2o_c,<NUM_ILE>*M_ile_L_c_tRNA_c+<NUM_ILE>*M_amp_c+<NUM_ILE>*M_ppi_c,0,inf
tRNA_c_charging_M_leu_L_c_<PATHWAY>,<NUM_LEU>*M_leu_L_c+<NUM_LEU>*M_atp_c+<NUM_LEU>*tRNA_c+<NUM_LEU>*M_h2o_c,<NUM_LEU>*M_leu_L_c_tRNA_c+<NUM_LEU>*M_amp_c+<NUM_LEU>*M_ppi_c,0,inf
tRNA_c_charging_M_lys_L_c_<PATHWAY>,<NUM_LYS>*M_lys_L_c+<NUM_LYS>*M_atp_c+<NUM_LYS>*tRNA_c+<NUM_LYS>*M_h2o_c,<NUM_LYS>*M_lys_L_c_tRNA_c+<NUM_LYS>*M_amp_c+<NUM_LYS>*M_ppi_c,0,inf
tRNA_c_charging_M_met_L_c_<PATHWAY>,<NUM_MET>*M_met_L_c+<NUM_MET>*M_atp_c+<NUM_MET>*tRNA_c+<NUM_MET>*M_h2o_c,<NUM_MET>*M_met_L_c_tRNA_c+<NUM_MET>*M_amp_c+<NUM_MET>*M_ppi_c,0,inf
tRNA_c_charging_M_phe_L_c_<PATHWAY>,<NUM_PHE>*M_phe_L_c+<NUM_PHE>*M_atp_c+<NUM_PHE>*tRNA_c+<NUM_PHE>*M_h2o_c,<NUM_PHE>*M_phe_L_c_tRNA_c+<NUM_PHE>*M_amp_c+<NUM_PHE>*M_ppi_c,0,inf
tRNA_c_charging_M_pro_L_c_<PATHWAY>,<NUM_PRO>*M_pro_L_c+<NUM_PRO>*M_atp_c+<NUM_PRO>*tRNA_c+<NUM_PRO>*M_h2o_c,<NUM_PRO>*M_pro_L_c_tRNA_c+<NUM_PRO>*M_amp_c+<NUM_PRO>*M_ppi_c,0,inf
tRNA_c_charging_M_ser_L_c_<PATHWAY>,<NUM_SER>*M_ser_L_c+<NUM_SER>*M_atp_c+<NUM_SER>*tRNA_c+<NUM_SER>*M_h2o_c,<NUM_SER>*M_ser_L_c_tRNA_c+<NUM_SER>*M_amp_c+<NUM_SER>*M_ppi_c,0,inf
tRNA_c_charging_M_thr_L_c_<PATHWAY>,<NUM_THR>*M_thr_L_c+<NUM_THR>*M_atp_c+<NUM_THR>*tRNA_c+<NUM_THR>*M_h2o_c,<NUM_THR>*M_thr_L_c_tRNA_c+<NUM_THR>*M_amp_c+<NUM_THR>*M_ppi_c,0,inf
tRNA_c_charging_M_trp_L_c_<PATHWAY>,<NUM_TRP>*M_trp_L_c+<NUM_TRP>*M_atp_c+<NUM_TRP>*tRNA_c+<NUM_TRP>*M_h2o_c,<NUM_TRP>*M_trp_L_c_tRNA_c+<NUM_TRP>*M_amp_c+<NUM_TRP>*M_ppi_c,0,inf
tRNA_c_charging_M_tyr_L_c_<PATHWAY>,<NUM_TYR>*M_tyr_L_c+<NUM_TYR>*M_atp_c+<NUM_TYR>*tRNA_c+<NUM_TYR>*M_h2o_c,<NUM_TYR>*M_tyr_L_c_tRNA_c+<NUM_TYR>*M_amp_c+<NUM_TYR>*M_ppi_c,0,inf
tRNA_c_charging_M_val_L_c_<PATHWAY>,<NUM_VAL>*M_val_L_c+<NUM_VAL>*M_atp_c+<NUM_VAL>*tRNA_c+<NUM_VAL>*M_h2o_c,<NUM_VAL>*M_val_L_c_tRNA_c+<NUM_VAL>*M_amp_c+<NUM_VAL>*M_ppi_c,0,inf
"""

In [ ]:
import Bio.PDB.Polypeptide
from Bio.Seq import Seq
from collections import Counter

In [ ]:
print output_trna_charging_rxns(reduce(lambda x, acc: x + '_' + acc, names), sequences)

In [ ]:
strs = ["R_alaAC::M_pyr_c+M_glu_L_c --> M_ala_L_c+M_akg_c",                                                                                                                                      
"R_alaAC_reverse::M_ala_L_c+M_akg_c --> M_pyr_c+M_glu_L_c",                                                                                                                                 
"R_arg::M_accoa_c+2.0*M_glu_L_c+3.0*M_atp_c+M_nadph_c+M_h_c+M_h2o_c+M_nh3_c+M_co2_c+M_asp_L_c --> M_coa_c+2.0*M_adp_c+2.0*M_pi_c+M_nadp_c+M_akg_c+M_ac_c+M_amp_c+M_ppi_c+M_fum_c+M_arg_L_c",
"R_aspC::M_glu_L_c+M_oaa_c --> M_asp_L_c+M_akg_c",                                                                                                                                          
"R_asnB::M_asp_L_c+M_gln_L_c+M_h2o_c+M_atp_c --> M_asn_L_c+M_glu_L_c+M_ppi_c+M_amp_c"   ,                                                                                                   
"R_asnA::M_asp_L_c+M_atp_c+M_nh3_c --> M_asn_L_c+M_ppi_c+M_amp_c"                 ,                                                                                                         
"R_cysEMK::M_ser_L_c+M_accoa_c+M_h2s_c --> M_cys_L_c+M_coa_c+M_ac_c"                ,                                                                                                       
"R_gltBD::M_gln_L_c+M_akg_c+M_nadph_c+M_h_c --> 2.0*M_glu_L_c+M_nadp_c"               ,                                                                                                     
"R_gdhA::M_akg_c+M_nadph_c+M_nh3_c+M_h_c --> M_glu_L_c+M_h2o_c+M_nadp_c"               ,                                                                                                    
"R_gdhA_reverse::M_glu_L_c+M_h2o_c+M_nadp_c --> M_akg_c+M_nadph_c+M_nh3_c+M_h_c"         ,                                                                                                  
"R_glnA::M_glu_L_c+M_atp_c+M_nh3_c --> M_gln_L_c+M_adp_c+M_pi_c"       ,                                                                                                                    
"R_glyA::M_ser_L_c+M_thf_c --> M_gly_L_c+M_h2o_c+M_mlthf_c"         ,                                                                                                                       
"R_his::M_gln_L_c+M_r5p_c+2.0*M_atp_c+2.0*M_nad_c+3.0*M_h2o_c --> M_his_L_c+M_akg_c+M_aicar_c+2.0*M_nadh_c+M_amp_c+M_pi_c+2.0*M_ppi_c+2.0*M_h_c"     ,                                      
"R_ile::M_thr_L_c+M_h_c+M_pyr_c+M_nadph_c+M_glu_L_c --> M_ile_L_c+M_h2o_c+M_nh3_c+M_co2_c+M_nadp_c+M_akg_c"     ,                                                                           
"R_leu::2.0*M_pyr_c+M_glu_L_c+M_nad_c+M_nadph_c+M_accoa_c --> M_leu_L_c+2.0*M_co2_c+M_nadp_c+M_coa_c+M_nadh_c+M_akg_c"                ,                                                     
"R_lys::M_asp_L_c+M_atp_c+2.0*M_nadph_c+2.0*M_h_c+M_pyr_c+M_succoa_c+M_glu_L_c --> M_lys_L_c+M_adp_c+M_pi_c+2.0*M_nadp_c+M_coa_c+M_akg_c+M_succ_c+M_co2_c"    ,                             
"R_met::M_asp_L_c+M_cys_L_c+M_succoa_c+M_atp_c+2.0*M_nadph_c+M_5mthf_c+M_h2o_c+2.0*M_h_c --> M_met_L_c+M_coa_c+M_succ_c+M_adp_c+M_pi_c+2.0*M_nadp_c+M_thf_c+M_nh3_c+M_pyr_c"   ,            
"R_phe::M_chor_c+M_glu_L_c --> M_phe_L_c+M_co2_c+M_h2o_c+M_akg_c"      ,                                                                                                                    
"R_pro::M_glu_L_c+M_atp_c+2.0*M_h_c+2.0*M_nadph_c --> M_pro_L_c+M_adp_c+2.0*M_nadp_c+M_pi_c+M_h2o_c"        ,                                                                               
"R_serABC::M_3pg_c+M_nad_c+M_glu_L_c+M_h2o_c --> M_ser_L_c+M_nadh_c+M_h_c+M_akg_c+M_pi_c"           ,                                                                                       
"R_thr::M_asp_L_c+2.0*M_atp_c+2.0*M_nadph_c+2.0*M_h_c+M_h2o_c --> M_thr_L_c+2.0*M_adp_c+2.0*M_pi_c+2.0*M_nadp_c"   ,                                                                        
"R_trp::M_chor_c+M_gln_L_c+M_ser_L_c+M_r5p_c+M_atp_c --> M_trp_L_c+M_glu_L_c+M_pyr_c+M_ppi_c+2.0*M_h2o_c+M_co2_c+M_g3p_c+M_amp_c"   ,                                                       
"R_tyr::M_chor_c+M_glu_L_c+M_nad_c --> M_tyr_L_c+M_akg_c+M_nadh_c+M_co2_c+M_h_c"                  ,                                                                                         
"R_val::2.0*M_pyr_c+M_h_c+M_nadph_c+M_glu_L_c --> M_val_L_c+M_co2_c+M_nadp_c+M_h2o_c+M_akg_c"]    

In [ ]:
for st in strs:
    rname = st.split('::')[0]
    print '"' + rname + '",'

In [ ]:
uptake = "[] --> M_<AA>_L_c"
secretion = "M_<AA>_L_c --> []"
aa_str = "^R_.*(-->.*M_<AA>_L_c)"
def gen_aas():
    for aa in Bio.PDB.Polypeptide.aa3:
        #print '"' + uptake.replace('<AA>', aa.lower()) + '",'
        #print '"' + secretion.replace('<AA>', aa.lower()) + '",'
        print 'r"' + aa_str.replace('<AA>', aa.lower()) + '",'
gen_aas()

In [ ]:
for name, sequence in zip(names, sequences):
    print output_translation_rxns(name, sequence)

src
Metabolite
"list_of_species":[
    {
        "species_symbol":"GENE_Crt1",
        "species_bound_type":"balanced",
        "species_compartment":"reactor"
    },
    dest
Gene
 {u'id': u'b0241', u'name': u'b0241'}
 Metabolite
 {u'charge': -2,
 u'compartment': u'c',
 u'formula': u'C20H21N7O7',
 u'id': u'10fthf_c',
 u'name': u'10-Formyltetrahydrofolate'}""

In [ ]:
""" 

Src
Reaction
"list_of_reactions":{
    "reaction_name":"R_glk_atp",
    "reaction_type_flag":"4770144144",
    "reaction_type":"solved",
    "catalyst_lexeme":"[]",
    "catalyst_ec_number":"[]",
    "list_of_reactants":[
        {
            "symbol":"M_atp_c",
            "stoichiometry":"1.0"
        },
        {
            "symbol":"M_glc_D_c",
            "stoichiometry":"1.0"
        }
    ],
    "list_of_products":[
        {
            "symbol":"M_adp_c",
            "stoichiometry":"1.0"
        },
        {
            "symbol":"M_g6p_c",
            "stoichiometry":"1.0"
        }
    ]
}
Dest
Reaction
{u'gene_reaction_rule': u'',
 u'id': u'DM_4CRSOL',
 u'lower_bound': 0.0,
 u'metabolites': {u'4crsol_c': -1.0},
 u'name': u'Sink needed to allow p-Cresol to leave system',
 u'upper_bound': 1000.0}

 """

In [ ]:
a = {}
a.update({'hi': 1})

In [ ]:
a

In [ ]:
def convert_rxn(julia_rxn, debug_dict):
    rxn_name = julia_rxn['reaction_name']
    debug_entry = debug_dict[rxn_name]
    metabs = {react["symbol"]: int(-1 * float(react["stoichiometry"])) for react in julia_rxn["list_of_reactants"]}
    metabs.update({product["symbol"]: int(float(product["stoichiometry"])) for product in julia_rxn["list_of_products"]})
    cobra_rxn = {
        'id': rxn_name,
        'name': rxn_name,
        'lower_bound': debug_entry['lower_bound'],
        'upper_bound': debug_entry['upper_bound'],
        'metabolites': metabs
    }
    return cobra_rxn

In [ ]:
def convert_metab(julia_metab, atom_dict):
    sym = julia_metab['species_symbol']
    cobra_metab = {
        'compartment': 'c',#julia_metab['species_compartment'],
        'id': sym,
        'name': sym,
        'formula': atom_dict[sym] if sym in atom_dict else None
    }
    return cobra_metab

In [ ]:
def parse_atom_file(f_atom):
    atom_dict = {}
    with open(f_atom, 'r') as f:
        for line in f.readlines():
            l = line.strip().split(',')
            if len(l) < 7:
                continue
            metab_name = l[0]
            metab_formula = 'C{0}H{1}N{2}O{3}P{4}S{5}'.format(*[l[i] for i in range(1, 7)])
            #C,H,N,O,P,S
            atom_dict[metab_name] = metab_formula
    return atom_dict

def parse_bounds(f_bounds, rxn_dict, idx_map):
    with open(f_bounds, 'r') as f_bounds:
        if f_bounds == 'DataDictionary.jl':
            bounds = False
            for i, l in enumerate(f_bounds.readlines()):
                line = l.strip().split() 
                if '];' in line:
                    bounds = False
                if bounds:
                    lb, ub = float(line[0]), float(line[1])
                    idx = int(line[6]) - 1
                    rxn_name = idx_map[i]
                    rxn_dict[rxn_name]['lower_bound'] = lb 
                    rxn_dict[rxn_name]['upper_bound'] = ub
                if 'default_bounds_array' in line:
                    bounds = True
        else:
            for idx, l in enumerate(f_bounds.readlines()):
                line = l.strip().split('\t') 
                lb, ub = map(float, line)
                rxn_name = idx_map[idx]
                rxn_dict[rxn_name]['lower_bound'] = lb 
                rxn_dict[rxn_name]['upper_bound'] = ub
    return rxn_dict
            
def parse_debug_file(f_debug, f_bounds):
    rxn_dict = {}
    idx_map = {}
    with open(f_debug, 'r') as f_debug:
        for line in f_debug.readlines():
            try:
                idx, rxn = line.strip().split(' ', 1)
            except ValueError:
                break
            rxn_name, rxn_str = rxn.split('::')
            # Julia is 1 indexed
            idx_map[int(idx) - 1] = rxn_name
            rxn_dict[rxn_name] = {'formula': rxn_str}
    rxn_dict = parse_bounds(f_bounds, rxn_dict, idx_map)
    return rxn_dict

In [ ]:
# Cats [u'reactions', u'genes', u'compartments', u'metabolites', u'version', u'id']
import json
import os
def julia_to_cobra_json(folder, fout=None):
    f_atom = os.path.join(folder, 'Atom.txt')
    atom_dict = parse_atom_file(f_atom)
    f_debug = os.path.join(folder, 'Debug.txt')
    f_bounds = os.path.join(folder, 'Bounds.txt') #DataDictionary.jl#Bounds.txt
    rxn_dict = parse_debug_file(f_debug, f_bounds)
    f_model = os.path.join(folder, 'Reactions.json')
    with open(f_model, 'r') as f:
        julia_model = json.load(f)

    cobra_model = {'id': 'cell_free_varner'}
    cobra_metabs, cobra_rxns = [], []
    cmpts = set()
    for julia_metab in julia_model["list_of_species"]:
        cobra_metab = convert_metab(julia_metab, atom_dict)
        cmpts.add(cobra_metab['compartment'].lower())
        cobra_metabs.append(cobra_metab)
    for julia_rxn in julia_model["list_of_reactions"]:
        cobra_rxn = convert_rxn(julia_rxn, rxn_dict)
        cobra_rxns.append(cobra_rxn)
    cobra_model['metabolites'] = cobra_metabs
    cobra_model['reactions'] = cobra_rxns
    cobra_model['genes'] = []
    cobra_cmpts = {}
    for cmpt in cmpts:
        cobra_cmpts[cmpt[0]] = 'cytosol'#cmpt.capitalize()
    cobra_model['compartments']  = cobra_cmpts
    if fout:
        with open(fout, 'w') as outfile:
            json.dump(cobra_model, outfile)
    return cobra_model
julia_to_cobra_json('../3rd_Party_Code/Sequence-Specific-FBA-CFPS-Publication-Code/Staging/', '../models/varner.json')